In [11]:
import asyncio
import aiohttp
import pandas as pd
from utils_warehouse import load_api_tokens, create_insert_table_db_async
from datetime import datetime



today = (datetime.now()).strftime('%Y-%m-%d')
semaphore = asyncio.Semaphore(10)

async def re_shipment_info_get(account, api_token):
    url = "https://marketplace-api.wildberries.ru/api/v3/supplies/orders/reshipment"
    headers = {"Authorization": api_token}
    
    async with semaphore:
        async with aiohttp.ClientSession(headers=headers) as session:
            retry_count = 0
            while retry_count < 5:
                try:
                    async with session.get(url) as response:
                        print(f"HTTP статус: {response.status} по ЛК {account}")
                        
                        if response.status == 400:
                            err = await response.json()
                            print(f"Ошибка 400 {account}: {err.get('message') or err}")
                            break  # Выходим из цикла, т.к. 400 обычно не исправляется повтором
                            
                        if response.status == 429:
                            print(f"429 Too Many Requests для {account} — ждём минуту")
                            retry_count += 1
                            await asyncio.sleep(60)
                            continue
                            
                        response.raise_for_status()
                        res_data = await response.json()
                        
                        # Добавляем account к каждому элементу
                        if isinstance(res_data, list):
                            for item in res_data:
                                # Добавляем account к основному объекту
                                item['account'] = account
                                
                                # Добавляем account к каждому заказу внутри orders
                                if 'orders' in item and isinstance(item['orders'], list):
                                    for order in item['orders']:
                                        order['account'] = account
                                        
                            return res_data
                        else:
                            # Если ответ не список, добавляем account к объекту
                            res_data['account'] = account
                            
                            # И к orders если они есть
                            if 'orders' in res_data and isinstance(res_data['orders'], list):
                                for order in res_data['orders']:
                                    order['date'] = today
                                    order['account'] = account
                                    
                            return res_data  # Возвращаем как список для единообразия
                            
                except aiohttp.ClientError as e:
                    print(f"Сетевая ошибка для {account}: {e}")
                    retry_count += 1
                    await asyncio.sleep(30)
            
            # Если дошли сюда - все попытки исчерпаны
            print(f"Не удалось получить данные для {account} после 5 попыток")
            return []
        
async def main():
    tasks = []
    for account, api_token in load_api_tokens().items():
        task = re_shipment_info_get(account, api_token)
        tasks.append(task)    
    all_data = await asyncio.gather(*tasks)
    processed_data = []
    for i in all_data:
        orders = i['orders']
        for order in orders:
            if order and order != []:  # Проверяем, что order не пустой
                processed_data.append(order)

        # Создаем один DataFrame из всех данных
        if processed_data:
            df_orders = pd.DataFrame(processed_data)
        else:
            print("Нет данных для создания DataFrame")
        df_orders = df_orders.rename(columns={'supplyId' : 'supply_id',
                                              'orderId' : 'order_id'})
        
    return df_orders

df = await main()

columns_type = {'supply_id' : 'TEXT',
                'order_id' : 'BIGINT',
                'date' : 'DATE',
                'account' : 'TEXT'}
key_columns = ('order_id', 'account')
# Перед вставкой в БД нужно привести столбец date в тип datetime.date
df['date'] = pd.to_datetime(df['date']).dt.date
table_name = 're_shipments'
await(create_insert_table_db_async(df, table_name, columns_type, key_columns))

HTTP статус: 200 по ЛК Вектор2
HTTP статус: 200 по ЛК Старт2
HTTP статус: 200 по ЛК Лопатина
HTTP статус: 200 по ЛК Пилосян
HTTP статус: 200 по ЛК Хачатрян
HTTP статус: 200 по ЛК Даниелян
HTTP статус: 200 по ЛК Вектор
HTTP статус: 200 по ЛК Тоноян
HTTP статус: 200 по ЛК Старт
HTTP статус: 200 по ЛК Оганесян
